In [1]:
!pip install -e ..

Obtaining file:///home/jhoskins/Development/casa/astrohack
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for astrohack (pyproject.toml) ... done
  Created wheel for astrohack: filename=astrohack-0.0.1-py3-none-any.whl size=1045 sha256=f276b572ef50ec243cd6718b68da3cac5f106d3995041e3f47771a9daac98464
  Stored in directory: /tmp/pip-ephem-wheel-cache-kayfveb2/wheels/45/b9/76/5b24db6479cc2a68e968851c684c14c29476c37b6e647b698d
Successfully built astrohack
  Attempting uninstall: astrohack
    Found existing installation: astrohack 0.0.1
    Uninstalling astrohack-0.0.1:
      Successfully uninstalled astrohack-0.0.1


In [2]:
import os
import gdown
import tarfile
import shutil
import dask
import requests
import time
import psutil

import numpy as np
import matplotlib.pyplot as plt

from astrohack import holog
from astrohack.dio import extract_holog
from astrohack.dio import load_hack_file
from astrohack._utils._io import _make_ant_pnt_dict

from dask.distributed import Client, LocalCluster

plt.rcParams["figure.figsize"] = (15, 15)

In [3]:
def performace(func):
    def wrapper(*args, **kwargs):
        start = time.time()
        func(*args, **kwargs)
        stop = time.time()
        print("Completion time: {}".format(stop - start))
    return wrapper

def get_max_memory_allocation():
    memory_string = str(int((psutil.virtual_memory().total/1e9)/os.cpu_count()))
    return "".join((memory_string, 'GB'))

def get_example_data():
    google_file_id = '1zpOtduyXtbh0wg0s5KWYuEMy7roGq92a'
    output = 'alma_band3.calibrated.DV16.ms.tar'
    path = "/".join((os.getcwd(), output))
    
    download_file_from_google_drive(
        id=google_file_id,
        destination=path
    )

    
    tar = tarfile.open(path)
    tar.extractall()
    tar.close()
    
    os.remove(path)
    
    return path.split('.tar')[0]



def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id , 'confirm': 1 }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [4]:
#ms_name = get_example_data()
ms_name = '/home/jhoskins/Development/casa/astrohack/examples/alma_band3.calibrated.DV16.ms'

In [5]:
cluster = LocalCluster(
    n_workers=6,
    threads_per_worker=1,
    memory_limit='8GB'
)

client = Client(cluster)
print(client.dashboard_link)

2022-12-02 15:56:18,597 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-jw6qyo9z', purging
2022-12-02 15:56:18,597 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-m7_cllul', purging
2022-12-02 15:56:18,597 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-gz3z4box', purging
2022-12-02 15:56:18,597 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-7rr46bhf', purging
2022-12-02 15:56:18,597 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-dtxgrr4w', purging
2022-12-02 15:56:18,597 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space/worker-ssdx19i4', purging


http://127.0.0.1:8787/status


In [6]:
DA = ['DA41', 'DA42', 'DA43', 'DA44', 'DA45', 'DA46', 'DA48', 'DA49', 'DA50', 'DA51', 'DA52', 'DA53', 'DA54', 'DA55', 'DA56', 'DA57', 'DA58', 'DA59']
DV = ['DV02', 'DV03', 'DV04', 'DV11', 'DV12', 'DV13', 'DV14', 'DV15', 'DV16', 'DV17', 'DV18', 'DV19', 'DV20', 'DV21', 'DV22', 'DV23', 'DV24', 'DV25'] 

holog_obs_description = {
    0:{
        2:{
            'map':DV,
            'ref':DA
        } 
    }
}

In [7]:
holog(
    ms_name=ms_name, 
    hack_name='hack.dict', 
    holog_obs_description=holog_obs_description,
    data_col='DATA',
    subscan_intent='MIXED',
    parallel=True
)

client.get_events("runtimes")

Successful readonly open of usernoread-locked table /home/jhoskins/Development/casa/astrohack/examples/alma_band3.calibrated.DV16.ms/ANTENNA: 8 columns, 36 rows
/home/jhoskins/Development/casa/astrohack/examples/alma_band3.calibrated.DV16.ms/DATA_DESCRIPTION
Processing ddi: 0, scan: 2


((1670014588.9529727,
  {'ant': 19,
   'hackfile': 'hack.dict',
   'message': 'h.a.c.k. ....',
   'worker': 'tcp://127.0.0.1:34309'}),
 (1670014588.9533622,
  {'ant': 9,
   'hackfile': 'hack.dict',
   'message': 'h.a.c.k. ....',
   'worker': 'tcp://127.0.0.1:38623'}),
 (1670014588.9534888,
  {'ant': 26,
   'hackfile': 'hack.dict',
   'message': 'h.a.c.k. ....',
   'worker': 'tcp://127.0.0.1:36519'}),
 (1670014588.9537094,
  {'ant': 31,
   'hackfile': 'hack.dict',
   'message': 'h.a.c.k. ....',
   'worker': 'tcp://127.0.0.1:43665'}),
 (1670014588.953801,
  {'ant': 21,
   'hackfile': 'hack.dict',
   'message': 'h.a.c.k. ....',
   'worker': 'tcp://127.0.0.1:45759'}),
 (1670014588.9539847,
  {'ant': 10,
   'hackfile': 'hack.dict',
   'message': 'h.a.c.k. ....',
   'worker': 'tcp://127.0.0.1:42995'}),
 (1670014588.9557447,
  {'ant': 28,
   'hackfile': 'hack.dict',
   'message': 'h.a.c.k. ....',
   'worker': 'tcp://127.0.0.1:34309'}),
 (1670014588.95584,
  {'ant': 2,
   'hackfile': 'hack.dic

In [ ]:
hack = load_hack_file('hack.dict')

In [ ]:
time = hack[0][2][26].time.values
l = hack[0][2][26].DIRECTIONAL_COSINES[:, 0]
m = hack[0][2][26].DIRECTIONAL_COSINES[:, 1]

time

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=1)

ax[0].scatter(l, m)
ax[0].set_xlabel('DIRECTIONAL COSINE(L)')
ax[0].set_ylabel('DIRECTIONAL COSINE(M)')

ax[1].scatter(time, l, label='L')
ax[1].scatter(time, m, label='M')

ax[1].set_xlabel('time')
ax[1].set_ylabel('DIRECTIONAL COSINES(L, M)')

ax[1].legend(bbox_to_anchor=(1.02, 1.0), loc='upper left', borderaxespad=0.)

plt.show()